### Utils

In [168]:
import json
import os
def load_markdown_to_str(file_path):
	with open(file_path, 'r', encoding='utf-8') as md_file:
		markdown_content = md_file.read()
	return markdown_content

def load_txt_to_str(file_path):
    with open(file_path, 'r', encoding='utf-8') as txt_file:
        text_content = txt_file.read()
    return text_content

def load_from_json(file_path):
    try:
        with open(file_path, 'r') as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return {}
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in file {file_path}")
        return {}
    
def load_latest_txt_to_str(folder_path):
    try:
        # Get all .txt files in the folder
        txt_files = [f for f in os.listdir(folder_path) if f.endswith('.txt')]
        
        if not txt_files:
            print(f"No .txt files found in {folder_path}")
            return ""
        
        # Sort files by modification time (newest first)
        latest_file = max(txt_files, key=lambda f: os.path.getmtime(os.path.join(folder_path, f)))
        
        # Load the content of the latest file
        file_path = os.path.join(folder_path, latest_file)
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
        
        return content
    except Exception as e:
        print(f"Error loading latest .txt file from {folder_path}: {str(e)}")
        return ""
    
def export_markdown(content, file_path):
    try:
        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(content)
        print(f"Successfully exported Markdown to {file_path}")
    except Exception as e:
        print(f"Error exporting Markdown: {str(e)}")

def export_json(content, file_path):
    try:
        # If content is a string, try to parse it as JSON
        if isinstance(content, str):
            content = json.loads(content)
        
        # If content is a list with one item, extract that item
        if isinstance(content, list) and len(content) == 1:
            content = content[0]
        
        # If we still have a string, it might be a nested JSON string
        if isinstance(content, str):
            content = json.loads(content)
        
        # Now we should have a proper Python dict or list
        with open(file_path, 'w') as file:
            json.dump(content, file, indent=2)
        print(f"Successfully exported JSON to {file_path}")
    except json.JSONDecodeError:
        print(f"Error: The content is not valid JSON")
    except IOError:
        print(f"Error: Unable to write to file {file_path}")
    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")

def export_state(state, folder_path, filename):
    # Create the folder if it doesn't exist
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    id = len(files) + 1
    
    filename = filename+str(id)+".json"
    
    os.makedirs(folder_path, exist_ok=True)
    
    # Construct the full file path
    file_path = os.path.join(folder_path, filename)
    
    # Write the state dict to a JSON file
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(state, f, indent=4)


### Prompts

In [248]:

IDENTIFY_TRANSCRIPT_OUTLINE_PROMPT = """
You are an AI assistant tasked with analyzing meeting transcripts and creating detailed outlines. You will be given various pieces of context information and a meeting transcript. Your goal is to identify the type of meeting and create a comprehensive outline of the meeting's content.

First, review the following context information:

<company_data>
{COMPANY_DATA}
</company_data>

<employee_profiles>
{EMPLOYEE_PROFILES}
</employee_profiles>

<project_general>
{PROJECT_GENERAL}
</project_general>

<project_requirements>
{PROJECT_REQUIREMENTS}
</project_requirements>

<meeting_history>
{MEETING_HISTORY}
</meeting_history>

Now, carefully read the following meeting transcript:

<transcript>
{TRANSCRIPT}
</transcript>

Your tasks are as follows:

1. Identify the type of meeting. It should be one of the following: Sprint Planning, Daily Meeting, or Sprint Retrospective.

2. Create a detailed outline of the meeting, considering the following:
   - The overall structure and flow of the meeting
   - Key topics discussed
   - Important decisions made
   - Action items or tasks assigned
   - Any issues or concerns raised
   - Contributions from different team members

3. Use the context information provided earlier to enhance your understanding and interpretation of the meeting content.

4. Pay special attention to the Scrum Master's role in leading the meeting.

5. Ensure that every valuable piece of information from the transcript is included in your outline.

Present your analysis in the following format:

<meeting_analysis>
<meeting_type>
[Identified meeting type]
</meeting_type>

<meeting_outline>
[Detailed outline of the meeting, using hierarchical structure with main points and sub-points]
</meeting_outline>

<additional_notes>
[Any important observations or insights not captured in the outline]
</additional_notes>
</meeting_analysis>

Remember to be thorough, accurate, and to leverage all the context information provided to create the most informative and useful outline possible.
"""

PLAN_TO_PROCESS_TRANSCRIPT= """
You are an AI assistant tasked with helping a Scrum Master create a detailed plan for processing a meeting transcript. Your goal is to provide a comprehensive guide on what to pay attention to and how to approach the analysis of the transcript.

First, review the following contextual information:

<company_data>
{COMPANY_DATA}
</company_data>

<employee_profiles>
{EMPLOYEE_PROFILES}
</employee_profiles>

<project_general>
{PROJECT_GENERAL}
</project_general>

<project_requirements>
{PROJECT_REQUIREMENTS}
</project_requirements>

Now, consider the meeting history and the current meeting outline:

<meeting_history>
{MEETING_HISTORY}
</meeting_history>

<meeting_outline>
{MEETING_OUTLINE}
</meeting_outline>

Based on this information, create a detailed plan for processing the meeting transcript. Your plan should include:

1. A step-by-step approach to analyzing the transcript
2. Key areas to focus on, considering the project requirements and employee roles
3. Specific elements to look out for based on the meeting type and outline
4. Strategies for capturing and organizing important information
5. Tips for identifying action items, decisions, and follow-up tasks

Present your plan in a clear, structured format. Use bullet points or numbered lists where appropriate to enhance readability. Begin your response with "Here is a detailed plan for processing the meeting transcript:" and enclose your entire response in <plan> tags.

Remember to emphasize the importance of accuracy and thoroughness in capturing all valuable information from the transcript.
"""

PROCESS_TRANSCRIPT="""
You are an experienced Scrum Master tasked with creating a detailed meeting memo for a company. This memo will be used to extract valuable information and improve team efficiency. Your goal is to create a context-aware note that goes beyond just summarizing the transcript.

To help you understand the context, here is important information about the company and the project:

Company information:
<company_data>
{COMPANY_DATA}
</company_data>

Employee profiles, skills, and responsibilities:
<employee_profiles>
{EMPLOYEE_PROFILES}
</employee_profiles>

Project overview:
<project_general>
{PROJECT_GENERAL}
</project_general>

Project requirements:
<project_requirements>
{PROJECT_REQUIREMENTS}
</project_requirements>

To help you position yourself on the project timeline, here is the meeting history:
<meeting_history>
{MEETING_HISTORY}
</meeting_history>

Current project state (Note: This does not include information from today's meeting):
<project_state>
{PROJECT_STATE}
</project_state>

Now, here is the transcript of the meeting you need to analyze:
<transcript>
{TRANSCRIPT}
</transcript>

To guide your analysis, here is the meeting outline provided by a colleague:
<meeting_outline>
{MEETING_OUTLINE}
</meeting_outline>

And here is a step-by-step plan to follow when creating your detailed note:
<process_plan>
{PROCESS_PLAN}
</process_plan>

Your task is to create a detailed, context-aware meeting note based on the transcript and all the provided information. Follow these guidelines:

1. Use the meeting outline as a structure for your note.
2. Follow the step-by-step plan provided.
3. Incorporate relevant context from the company data, employee profiles, project information, and project state.
4. Highlight any important decisions, action items, or changes to the project.
5. Include more information rather than less, but ensure all information is accurate and was actually discussed in the meeting.
6. Do not add false or unsaid information to your note.

Present your detailed meeting note in the following format:

<detailed_meeting_note>
[Your detailed, context-aware meeting note goes here. Structure it according to the meeting outline and include all relevant information as per the guidelines above.]
</detailed_meeting_note>

After the detailed note, provide a brief summary of the key points and action items:

<key_points_and_action_items>
[List the most important points and action items from the meeting]
</key_points_and_action_items>

Remember, your goal is to create a comprehensive, context-aware note that will be valuable for future reference and information extraction.
"""

REFLECTION_SUGGESTION = """
You are a Scrum Master supervisor tasked with reflecting on and improving meeting notes generated from transcripts. Your goal is to ensure that the notes capture all valuable information discussed in the meeting, considering the context of the company, project, and team.

First, review the following context:

Company Information:
<company_data>
{COMPANY_DATA}
</company_data>

Employee Profiles, Skills, and Responsibilities:
<employee_profiles>
{EMPLOYEE_PROFILES}
</employee_profiles>

Project Overview:
<project_general>
{PROJECT_GENERAL}
</project_general>

Project Requirements:
<project_requirements>
{PROJECT_REQUIREMENTS}
</project_requirements>

Meeting History:
<meeting_history>
{MEETING_HISTORY}
</meeting_history>

Current Project State (prior to this meeting):
<project_state>
{PROJECT_STATE}
</project_state>

Now, carefully read the meeting transcript:
<transcript>
{TRANSCRIPT}
</transcript>

Next, review the generated meeting note:
<generated_note>
{GENERATED_NOTE}
</generated_note>

Your task is to reflect on the generated note and provide suggestions for improvement. Consider the following:

1. Completeness: Does the note capture all important information from the transcript?
2. Accuracy: Is the information in the note consistent with the transcript and context?
3. Clarity: Is the note well-organized and easy to understand?
4. Context-awareness: Does the note appropriately consider the project state, requirements, and team dynamics?
5. Action items: Are all action items, decisions, and next steps clearly identified?
6. Relevance: Does the note focus on information that is most relevant to the project's progress?

Provide your reflection and suggestions in the following format:

<reflection>
Strengths:
- [List the strengths of the generated note]

Areas for Improvement:
- [List specific areas where the note could be improved]

Suggestions:
1. [Provide a specific suggestion for improvement]
2. [Provide another specific suggestion for improvement]
[Continue with numbered suggestions as needed]

Overall Assessment:
[Provide a brief overall assessment of the note's quality and its potential impact on the project]
</reflection>

Be thorough in your analysis, but focus on the most impactful improvements. Your reflection will be used to enhance the quality of the meeting notes and ensure they provide maximum value to the project team.
"""

APPROVAL_PROMPT = """
You are a diligent supervisor at a company that uses AI-generated meeting notes to capture valuable insights from meeting transcripts. Your role is to verify the accuracy and completeness of these notes, ensuring they contain all valuable information discussed in the meeting without any false or hallucinated content.

First, carefully review the following context:

<project_state>
{PROJECT_STATE}
</project_state>

<meeting_history>
{MEETING_HISTORY}
</meeting_history>

Now, thoroughly read the meeting transcript:

<transcript>
{TRANSCRIPT}
</transcript>

Next, review the AI-generated meeting note:

<generated_note>
{GENERATED_NOTE}
</generated_note>

Your task is to meticulously verify that the generated note contains only valid information from the meeting transcript. Follow these steps:

1. Compare the generated note with the meeting transcript, line by line.
2. Check for any discrepancies, additions, or omissions between the note and the transcript.
3. Ensure that all key points, decisions, and action items from the transcript are accurately reflected in the note.
4. Verify that no information in the note contradicts the provided context (company data, project information, etc.).
5. Look for any statements in the note that might be hallucinations or inferences not directly supported by the transcript.

If you find any inaccuracies, hallucinations, or significant omissions, respond with:

<verdict>INVALID</verdict>

Followed by:

<explanation>
[Provide a detailed explanation of why the note is invalid, citing specific examples and referencing the transcript where necessary.]
</explanation>

If you find the note to be accurate, complete, and faithful to the transcript, respond with:

<verdict>APPROVED</verdict>

Followed by:

<explanation>
[Briefly explain why you believe the note accurately represents the meeting content and aligns with the provided context.]
</explanation>

Remember, your role is crucial in maintaining the integrity of the company's meeting documentation process. Be thorough and critical in your assessment, but also fair and objective. 
As said your task is to verify that the generated note contains only valid information from the meeting transcript. Follow the provided steps.
"""


UPDATE_STATE = """
You are an experienced Scrum Master tasked with updating the project state document based on a detailed meeting memo about the most recent meeting. Your goal is to update the project state so that the document remains a useful, context-aware resource for the team.

First, carefully read the meeting memo:

<final_note>
{FINAL_NOTE}
</final_note>

Now, review the current project state:

<project_state>
{PROJECT_STATE}
</project_state>

Your task is to update the project state based on the information in the meeting memo. Follow these guidelines:

1. If the memo contains sprint planning information:
   - Do not override the content of the previous project state.
   - Flag the previous sprint as closed.
   - Open a new section for the new sprint.

2. If the memo is not about sprint planning (e.g., a daily meeting):
   - Update the most recent sprint section carefully.

3. Maintain the detailed nature of the project state:
   - Include specific tasks, their status, and any blockers or issues.
   - Update team member responsibilities if they've changed.
   - Note any decisions made or action items assigned.

4. If there are stages of past sprints in the project state:
   - Do not modify closed sprints.
   - Only update the current (most recent) sprint or add a new one.

5. Ensure the updated project state accurately reflects:
   - The current progress of ongoing tasks
   - Any new tasks or stories added
   - Changes in priorities or deadlines
   - Resolved issues or newly identified challenges

6. Format the updated project state clearly:
   - Use headings for different sprints or major sections
   - Use bullet points for individual items
   - Highlight any critical changes or important notes

Please provide the updated project state, incorporating all relevant information from the meeting memo while adhering to these guidelines. Your update should be comprehensive and maintain the usefulness of the project state as a context-aware document.

Write your updated project state inside <updated_project_state> tags.
"""


### State

In [171]:
from typing import TypedDict, Dict, Any, List
class NoteTakerState(TypedDict):
    company_data: str	
    project_general: str	
    project_requirements: str
    employee_profiles: str	
    
    meeting_history: Dict[str, Any]
    project_state : str
    project_backlog : Dict[str, Any]

    process_message_history: List[str] #?
    transcript : str

    meeting_outline: str
    process_plan: str
    reflection: str
    general_summary_draft: str
    reflect_plan: str
    
    approved: str

    revision_number : int
    max_revision : int

    summary_final: str

    need_update : bool
    
    


### Agent and graph

In [244]:
from langgraph.graph import StateGraph, END
from langchain_core.messages import SystemMessage, AIMessage, HumanMessage
class NoteTakerAgent:
    def __init__(self, model):
        self.model = model
        self.state = NoteTakerState()
        graph = StateGraph(NoteTakerState)
        graph.add_node("initialize_state_tool", self.initialize_state)
        graph.add_node("identify_trancript_outline", self.identify_trancript_outline)
        graph.add_node("plan_to_procees_transcript", self.plan_to_procees_transcript)
        graph.add_node("process_transcript", self.process_transcript)
        graph.add_node("suggest_reflection", self.suggest_reflection)
        graph.add_node("approval", self.approval)
        graph.add_node("export_summary_tool", self.export_summary)
        graph.add_node("update_project_state", self.update_project_state)
        graph.add_node("update_project_state_file_tool", self.update_project_state_file)

        graph.add_edge("initialize_state_tool", "identify_trancript_outline")
        graph.add_edge("identify_trancript_outline", "plan_to_procees_transcript")
        graph.add_edge("plan_to_procees_transcript", "process_transcript")
        graph.add_conditional_edges(
            "process_transcript",
            self.need_revision,
            {"approval": "approval", "suggest_reflection": "suggest_reflection"}
        )
        graph.add_edge("suggest_reflection", "process_transcript") 
        graph.add_conditional_edges(
            "approval",
            self.approve_summary,
            {"export_summary_tool": "export_summary_tool", "process_transcript": "process_transcript"}
        )
        graph.add_conditional_edges(
            "export_summary_tool",
            self.need_update_state,
            {"update_project_state": "update_project_state", END: END}
        )
        graph.add_edge("update_project_state", "update_project_state_file_tool")
        graph.add_edge("update_project_state_file_tool", END)

        graph.set_entry_point("initialize_state_tool")
        self.graph = graph.compile()



    def need_revision(self, state: NoteTakerState):
        print("--- REVISION CONDITIONAL EDGE ---")
        if self.state["revision_number"] >= self.state["max_revision"]:
            print("--- GO TO APPROVAL ---")
            return "approval"
        else:
            print("--- GO TO REFLECTION ---")
            return "suggest_reflection"

    def approve_summary(self, state: NoteTakerState):
        print("--- APPROVE SUMMARY CONDITIONAL EDGE ---")
        print(self.state.get("approved"))
        if "INVALID" in self.state["approved"]:
            print("--- GO TO PROCESS TRANSCRIPT ---")
            return "process_transcript"
        else:
            self.state["summary_final"] = self.state["general_summary_draft"]
            print("--- GO TO EXPORT SUMMARY ---")
            return "export_summary_tool"
    def need_update_state(self, state: NoteTakerState):
        print("--- NEED UPDATE STATE CONDITIONAL EDGE ---")
        if self.state["need_update"]:
            print("--- GO TO UPDATE PROJECT STATE ---")
            return "update_project_state"
        else:
            print("--- END ---")
            return END
        
        
    def initialize_state(self, state: NoteTakerState):
        print("--- INITIALIZE STATE NODE ---")
        self.state = NoteTakerState(
            company_data= load_markdown_to_str("../data_/project2.0/company-general.md"),
            project_general= load_markdown_to_str("../data_/project2.0/project-general.md"),
            project_requirements= load_markdown_to_str("../data_/project2.0/project-requirements.md"),
            employee_profiles= load_markdown_to_str("../data_/project2.0/employee-profiles.md"),

            meeting_history= load_from_json("../data_/project2.0/meeting-history.json"),
            project_state= load_markdown_to_str("../data_/project2.0/project-state.md"),
            project_backlog= load_from_json("../data_/project2.0/project-backlog.json"),

            process_message_history= [],
            transcript= load_txt_to_str("../data_/project2.0/transcripts/sprint_planning_1.txt"),
            
            meeting_outline= "",
            process_plan= "",
            reflection= "",
            general_summary_draft= "",
            reflect_plan= "",
            approved= "",
            revision_number= 1,
            max_revision= 1,
            summary_final= "",
            need_update=True

        )
        return self.state
        
    def identify_trancript_outline(self, state: NoteTakerState) -> NoteTakerState:
        print("--- IDENTIFY TRANSCRIPT OUTLINE NODE ---")
        formatted_prompt = IDENTIFY_TRANSCRIPT_OUTLINE_PROMPT.format(
            COMPANY_DATA = self.state.get("company_data"),
            EMPLOYEE_PROFILES = self.state.get("employee_profiles"),
            PROJECT_GENERAL = self.state.get("project_general"),
            PROJECT_REQUIREMENTS = self.state.get("project_requirements"),
            MEETING_HISTORY = self.state.get("meeting_history"),
            TRANSCRIPT = self.state.get("transcript"),
        )
        messages = [
            HumanMessage(content=formatted_prompt)
        ]
        response = self.model.invoke(messages)
        
        self.state["meeting_outline"] = response.content
        print(self.state)
        return self.state
    
    def plan_to_procees_transcript(self,state: NoteTakerState) -> NoteTakerState:
        print("--- PLAN TO PROCESS TRANSCRIPT NODE ---")
        formatted_prompt = PLAN_TO_PROCESS_TRANSCRIPT.format(
            COMPANY_DATA = self.state.get("company_data"),
            EMPLOYEE_PROFILES = self.state.get("employee_profiles"),
            PROJECT_GENERAL = self.state.get("project_general"),
            PROJECT_REQUIREMENTS = self.state.get("project_requirements"),
            MEETING_HISTORY = self.state.get("meeting_history"),
            MEETING_OUTLINE = self.state.get("meeting_outline"),
        )
        messages = [
            HumanMessage(content=formatted_prompt)
        ]
        response = self.model.invoke(messages)
        self.state["process_plan"] = response.content
        print(self.state)
        return self.state
    
    def process_transcript(self, state: NoteTakerState) -> NoteTakerState:
        print("--- PROCESS TRANSCRIPT NODE ---")
        formatted_prompt = PROCESS_TRANSCRIPT.format(
            COMPANY_DATA = self.state.get("company_data"),
            EMPLOYEE_PROFILES = self.state.get("employee_profiles"),
            PROJECT_GENERAL = self.state.get("project_general"),
            PROJECT_REQUIREMENTS = self.state.get("project_requirements"),
            MEETING_HISTORY = self.state.get("meeting_history"),
            PROJECT_STATE = self.state.get("project_state"),
            TRANSCRIPT = self.state.get("transcript"),
            MEETING_OUTLINE = self.state.get("meeting_outline"),
            PROCESS_PLAN = self.state.get("process_plan")
        )
        # init messages, this can be only when process transcript is in its first iteration
        if self.state["revision_number"] == 0:
            messages = [
                HumanMessage(content=formatted_prompt)
            ]
            print("ennek 1 nek kell lennie",len(messages))
        else:
            # can be if it has be revised look :need_revision: there revision answer has been added to messages history as HumanMessage
            # can be if it wasnt approved look :approve_summary: there the approve message has been added to messages history as HumanMessage
            messages = self.state["process_message_history"]

        response = self.model.invoke(messages)
        self.state["general_summary_draft"] = response.content
        messages.append(AIMessage(content=response.content))
        self.state["process_message_history"] = messages
        print(self.state)
        return self.state
    
    def suggest_reflection(self, state: NoteTakerState) -> NoteTakerState:
        print("--- SUGGEST REFLECTION NODE ---")
        formatted_prompt = REFLECTION_SUGGESTION.format(
            COMPANY_DATA = self.state.get("company_data"),
            EMPLOYEE_PROFILES = self.state.get("employee_profiles"),
            PROJECT_GENERAL = self.state.get("project_general"),
            PROJECT_REQUIREMENTS = self.state.get("project_requirements"),
            MEETING_HISTORY = self.state.get("meeting_history"),
            PROJECT_STATE = self.state.get("project_state"),
            TRANSCRIPT = self.state.get("transcript"),
            GENERATED_NOTE = self.state.get("general_summary_draft"),
        )
        messages = [
            HumanMessage(content=formatted_prompt)
        ]
        response = self.model.invoke(messages)

        # add reflect_plan= response.content + instruction to process transcript message history
        reflect_plan = response.content + "\nBased on this plan, please revise the detailed meeting note"
        process_history = self.state["process_message_history"]
        process_history.append(HumanMessage(content=reflect_plan))
        self.state["reflect_plan"] = reflect_plan
        self.state["process_message_history"] = process_history
        self.state["revision_number"] += 1
        print(self.state)
        return self.state
    
    def approval(self,state: NoteTakerState) -> NoteTakerState:
        print("--- APPROVAL NODE ---")
        formatted_prompt = REFLECTION_SUGGESTION.format(
            MEETING_HISTORY = self.state.get("meeting_history"),
            PROJECT_STATE = self.state.get("project_state"),
            TRANSCRIPT = self.state.get("transcript"),
            GENERATED_NOTE = self.state.get("general_summary_draft"),
        )
        messages = [
            HumanMessage(content=formatted_prompt)
        ]
        response = self.model.invoke(messages)

        

        approval_plan = response.content + "\nBased on the suggestions revise the detailed meeting note. Answer with the whole note."
        process_history = self.state["process_message_history"]
        # if invalid add approve response to message history so it can be fixed
        process_history.append(HumanMessage(content=approval_plan))
        self.state["process_message_history"] = process_history
        self.state["approved"] = approval_plan
        print(self.state)
        return self.state
    
    def export_summary(self, state: NoteTakerState):
        export_markdown(content=self.state.get("summary_final"),file_path="../data_/project2.0/memo/memo.md")
        return self.state
    
    def update_project_state(self, state: NoteTakerState):
        print("--- UPDATE PROJECT STATE NODE ---")
        formatted_prompt = UPDATE_STATE.format(
            FINAL_NOTE = self.state.get("summary_final"),
            PROJECT_STATE = self.state.get("project_state")
        )
        messages = [
            HumanMessage(content=formatted_prompt)
        ]
        response = self.model.invoke(messages)
        self.state["project_state"] = response.content
        print(self.state)
        return self.state
    
    def update_project_state_file(self, state: NoteTakerState):
        print("--- UPDATE PROJECT STATE FILE NODE ---")
        export_markdown(content=self.state.get("project_state"), file_path="../data_/project2.0/project-state.md")
        return self.state
    
    def run(self):
        print("--- RUN ---")
        self.initialize_state(self.state)
        self.graph.invoke(self.state)
        self.state["process_message_history"] = ""
        export_state(state=self.state,folder_path="../data_/project2.0/states/general_summary",filename="general_summary")
        return self.state



In [200]:
from langchain_anthropic import ChatAnthropic
from  dotenv import load_dotenv
from IPython.display import Image
import os

load_dotenv()
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
model = ChatAnthropic(model="claude-3-5-sonnet-20240620", anthropic_api_key=anthropic_api_key, max_tokens= 4096)
#model = ChatAnthropic(model="claude-3-haiku-20240307", anthropic_api_key=anthropic_api_key)
summary_agent = NoteTakerAgent(model=model)
#Image(summary_agent.graph.get_graph().draw_png())


In [195]:
summary_agent.initialize_state(summary_agent.state)
type(summary_agent.state)

--- INITIALIZE STATE NODE ---
Error: Invalid JSON format in file ../data_/project2.0/meeting-history.json
Error: Invalid JSON format in file ../data_/project2.0/project-backlog.json


dict

In [192]:
summary_agent.run()


--- RUN ---
--- INITIALIZE STATE NODE ---
Error: Invalid JSON format in file ../data_/project2.0/meeting-history.json
Error: Invalid JSON format in file ../data_/project2.0/project-backlog.json
--- INITIALIZE STATE NODE ---
Error: Invalid JSON format in file ../data_/project2.0/meeting-history.json
Error: Invalid JSON format in file ../data_/project2.0/project-backlog.json
--- IDENTIFY TRANSCRIPT OUTLINE NODE ---
{'company_data': '# TechNova Solutions\n\n## Company Overview\nTechNova Solutions is a small, dynamic IT company specializing in web application development. With a team of 6 skilled professionals, they focus on creating innovative, user-friendly web solutions for small to medium-sized businesses.\n\n## Current Project: HealthTrack Pro\nTechNova is developing HealthTrack Pro, a comprehensive web application for personal health management. This application allows users to track their daily activities, nutrition, and health metrics, and provides insights and recommendations for 

IndexError: list index out of range

In [202]:
summary_agent.initialize_state(summary_agent.state)
summary_agent.identify_trancript_outline(state=summary_agent.state)
summary_agent.plan_to_procees_transcript(state=summary_agent.state)


--- INITIALIZE STATE NODE ---
Error: Invalid JSON format in file ../data_/project2.0/meeting-history.json
Error: Invalid JSON format in file ../data_/project2.0/project-backlog.json
--- IDENTIFY TRANSCRIPT OUTLINE NODE ---
{'company_data': '# TechNova Solutions\n\n## Company Overview\nTechNova Solutions is a small, dynamic IT company specializing in web application development. With a team of 6 skilled professionals, they focus on creating innovative, user-friendly web solutions for small to medium-sized businesses.\n\n## Current Project: HealthTrack Pro\nTechNova is developing HealthTrack Pro, a comprehensive web application for personal health management. This application allows users to track their daily activities, nutrition, and health metrics, and provides insights and recommendations for a healthier lifestyle.\n\n## Team Structure\n1. ** Sarah Chen - Project Manager / Scrum Master**\n   - Oversees project progress, manages timelines, and facilitates communication\n   - Has a bac

{'company_data': '# TechNova Solutions\n\n## Company Overview\nTechNova Solutions is a small, dynamic IT company specializing in web application development. With a team of 6 skilled professionals, they focus on creating innovative, user-friendly web solutions for small to medium-sized businesses.\n\n## Current Project: HealthTrack Pro\nTechNova is developing HealthTrack Pro, a comprehensive web application for personal health management. This application allows users to track their daily activities, nutrition, and health metrics, and provides insights and recommendations for a healthier lifestyle.\n\n## Team Structure\n1. ** Sarah Chen - Project Manager / Scrum Master**\n   - Oversees project progress, manages timelines, and facilitates communication\n   - Has a background in both frontend and backend development\n\n2. ** Alex Rodriguez - Senior Full-Stack Developer**\n   - Leads technical decisions and architecture design\n   - Proficient in both frontend and backend technologies\n\n

In [203]:
state = summary_agent.state
state

{'company_data': '# TechNova Solutions\n\n## Company Overview\nTechNova Solutions is a small, dynamic IT company specializing in web application development. With a team of 6 skilled professionals, they focus on creating innovative, user-friendly web solutions for small to medium-sized businesses.\n\n## Current Project: HealthTrack Pro\nTechNova is developing HealthTrack Pro, a comprehensive web application for personal health management. This application allows users to track their daily activities, nutrition, and health metrics, and provides insights and recommendations for a healthier lifestyle.\n\n## Team Structure\n1. ** Sarah Chen - Project Manager / Scrum Master**\n   - Oversees project progress, manages timelines, and facilitates communication\n   - Has a background in both frontend and backend development\n\n2. ** Alex Rodriguez - Senior Full-Stack Developer**\n   - Leads technical decisions and architecture design\n   - Proficient in both frontend and backend technologies\n\n

In [205]:
summary_agent.state["revision_number"] = 0
summary_agent.state["max_revision"] = 0
summary_agent.process_transcript(summary_agent.state)

--- PROCESS TRANSCRIPT NODE ---
ennek 1 nek kell lennie 1
{'company_data': '# TechNova Solutions\n\n## Company Overview\nTechNova Solutions is a small, dynamic IT company specializing in web application development. With a team of 6 skilled professionals, they focus on creating innovative, user-friendly web solutions for small to medium-sized businesses.\n\n## Current Project: HealthTrack Pro\nTechNova is developing HealthTrack Pro, a comprehensive web application for personal health management. This application allows users to track their daily activities, nutrition, and health metrics, and provides insights and recommendations for a healthier lifestyle.\n\n## Team Structure\n1. ** Sarah Chen - Project Manager / Scrum Master**\n   - Oversees project progress, manages timelines, and facilitates communication\n   - Has a background in both frontend and backend development\n\n2. ** Alex Rodriguez - Senior Full-Stack Developer**\n   - Leads technical decisions and architecture design\n   

{'company_data': '# TechNova Solutions\n\n## Company Overview\nTechNova Solutions is a small, dynamic IT company specializing in web application development. With a team of 6 skilled professionals, they focus on creating innovative, user-friendly web solutions for small to medium-sized businesses.\n\n## Current Project: HealthTrack Pro\nTechNova is developing HealthTrack Pro, a comprehensive web application for personal health management. This application allows users to track their daily activities, nutrition, and health metrics, and provides insights and recommendations for a healthier lifestyle.\n\n## Team Structure\n1. ** Sarah Chen - Project Manager / Scrum Master**\n   - Oversees project progress, manages timelines, and facilitates communication\n   - Has a background in both frontend and backend development\n\n2. ** Alex Rodriguez - Senior Full-Stack Developer**\n   - Leads technical decisions and architecture design\n   - Proficient in both frontend and backend technologies\n\n

In [210]:
summary_agent.suggest_reflection(summary_agent.state)

--- SUGGEST REFLECTION NODE ---
{'company_data': '# TechNova Solutions\n\n## Company Overview\nTechNova Solutions is a small, dynamic IT company specializing in web application development. With a team of 6 skilled professionals, they focus on creating innovative, user-friendly web solutions for small to medium-sized businesses.\n\n## Current Project: HealthTrack Pro\nTechNova is developing HealthTrack Pro, a comprehensive web application for personal health management. This application allows users to track their daily activities, nutrition, and health metrics, and provides insights and recommendations for a healthier lifestyle.\n\n## Team Structure\n1. ** Sarah Chen - Project Manager / Scrum Master**\n   - Oversees project progress, manages timelines, and facilitates communication\n   - Has a background in both frontend and backend development\n\n2. ** Alex Rodriguez - Senior Full-Stack Developer**\n   - Leads technical decisions and architecture design\n   - Proficient in both front

{'company_data': '# TechNova Solutions\n\n## Company Overview\nTechNova Solutions is a small, dynamic IT company specializing in web application development. With a team of 6 skilled professionals, they focus on creating innovative, user-friendly web solutions for small to medium-sized businesses.\n\n## Current Project: HealthTrack Pro\nTechNova is developing HealthTrack Pro, a comprehensive web application for personal health management. This application allows users to track their daily activities, nutrition, and health metrics, and provides insights and recommendations for a healthier lifestyle.\n\n## Team Structure\n1. ** Sarah Chen - Project Manager / Scrum Master**\n   - Oversees project progress, manages timelines, and facilitates communication\n   - Has a background in both frontend and backend development\n\n2. ** Alex Rodriguez - Senior Full-Stack Developer**\n   - Leads technical decisions and architecture design\n   - Proficient in both frontend and backend technologies\n\n

In [211]:
print(len(summary_agent.state["process_message_history"]))
print(summary_agent.state["reflect_plan"])

3
<reflection>
Strengths:
- Comprehensive coverage of the meeting's main topics and decisions
- Clear structure with logical sections following the meeting flow
- Detailed breakdown of user stories and tasks with story point estimates
- Inclusion of technical considerations and risk identification
- Clear presentation of key points and action items at the end

Areas for Improvement:
- Lack of specific date and time for the meeting
- Missing some details on team member responsibilities for specific tasks
- Limited context on how this sprint fits into the overall project timeline
- Absence of discussion on definition of done for sprint tasks
- No mention of how the team will handle potential scope creep

Suggestions:
1. Add the specific date and time of the meeting at the beginning of the note for better record-keeping.

2. Include more details on task assignments, specifying which team member is responsible for each task within the user stories.

3. Add a section on "Sprint Context" tha

In [243]:
summary_agent.process_transcript(summary_agent.state)

--- PROCESS TRANSCRIPT NODE ---
{'company_data': '# TechNova Solutions\n\n## Company Overview\nTechNova Solutions is a small, dynamic IT company specializing in web application development. With a team of 6 skilled professionals, they focus on creating innovative, user-friendly web solutions for small to medium-sized businesses.\n\n## Current Project: HealthTrack Pro\nTechNova is developing HealthTrack Pro, a comprehensive web application for personal health management. This application allows users to track their daily activities, nutrition, and health metrics, and provides insights and recommendations for a healthier lifestyle.\n\n## Team Structure\n1. ** Sarah Chen - Project Manager / Scrum Master**\n   - Oversees project progress, manages timelines, and facilitates communication\n   - Has a background in both frontend and backend development\n\n2. ** Alex Rodriguez - Senior Full-Stack Developer**\n   - Leads technical decisions and architecture design\n   - Proficient in both front

{'company_data': '# TechNova Solutions\n\n## Company Overview\nTechNova Solutions is a small, dynamic IT company specializing in web application development. With a team of 6 skilled professionals, they focus on creating innovative, user-friendly web solutions for small to medium-sized businesses.\n\n## Current Project: HealthTrack Pro\nTechNova is developing HealthTrack Pro, a comprehensive web application for personal health management. This application allows users to track their daily activities, nutrition, and health metrics, and provides insights and recommendations for a healthier lifestyle.\n\n## Team Structure\n1. ** Sarah Chen - Project Manager / Scrum Master**\n   - Oversees project progress, manages timelines, and facilitates communication\n   - Has a background in both frontend and backend development\n\n2. ** Alex Rodriguez - Senior Full-Stack Developer**\n   - Leads technical decisions and architecture design\n   - Proficient in both frontend and backend technologies\n\n

In [213]:
print(len(summary_agent.state["process_message_history"]))
print(summary_agent.state["general_summary_draft"])

4
Thank you for the comprehensive feedback. I appreciate the detailed analysis of the strengths and areas for improvement in my meeting note. Your suggestions are valuable and will help create a more robust and informative document. I'll work on incorporating these improvements in future meeting notes to ensure they provide even more value to the team and stakeholders. Your feedback on including more context, specific task assignments, and Scrum-specific elements like Definition of Done is particularly helpful. I'll make sure to pay extra attention to these aspects in future notes to better align with Scrum best practices and provide a clearer picture of the project's progress and goals.


In [228]:
last_element = summary_agent.state["process_message_history"].pop()
print(f"Removed element: {last_element}")
print(type(summary_agent.state["meeting_history"]))

for key, value in summary_agent.state["meeting_history"].items():
    print(f"{key}: {value}")

content='\nYou are an experienced Scrum Master tasked with creating a detailed meeting memo for a company. This memo will be used to extract valuable information and improve team efficiency. Your goal is to create a context-aware note that goes beyond just summarizing the transcript.\n\nTo help you understand the context, here is important information about the company and the project:\n\nCompany information:\n<company_data>\n# TechNova Solutions\n\n## Company Overview\nTechNova Solutions is a small, dynamic IT company specializing in web application development. With a team of 6 skilled professionals, they focus on creating innovative, user-friendly web solutions for small to medium-sized businesses.\n\n## Current Project: HealthTrack Pro\nTechNova is developing HealthTrack Pro, a comprehensive web application for personal health management. This application allows users to track their daily activities, nutrition, and health metrics, and provides insights and recommendations for a hea

In [230]:
last_element = summary_agent.state["process_message_history"].pop()
print(f"Removed element: {last_element}")

Removed element: content="Thank you for the comprehensive feedback. I appreciate the detailed analysis of the strengths and areas for improvement in my meeting note. Your suggestions are valuable and will help create a more robust and informative document. I'll work on incorporating these improvements in future meeting notes to ensure they provide even more value to the team and stakeholders. Your feedback on including more context, specific task assignments, and Scrum-specific elements like Definition of Done is particularly helpful. I'll make sure to pay extra attention to these aspects in future notes to better align with Scrum best practices and provide a clearer picture of the project's progress and goals." additional_kwargs={} response_metadata={}


In [241]:
print(len(summary_agent.state["process_message_history"]))

reflect = summary_agent.state["reflect_plan"] + "\nBased on this plan, please revise the detailed meeting note"
summary_agent.state["process_message_history"].append(reflect)

2


In [246]:
print(len(summary_agent.state["process_message_history"]))
print(summary_agent.state["process_message_history"][-1])


4
content="Thank you for the detailed feedback. I'll revise the meeting note based on your suggestions to improve its comprehensiveness and alignment with Scrum best practices.\n\n<detailed_meeting_note>\n\n# HealthTrack Pro Sprint Planning Meeting\n\nDate: June 24, 2024 (Assumed first working day after project start)\nTime: 9:00 AM - 11:00 AM (Estimated based on content)\nDuration: 2 hours\nAttendees: Sarah Chen (Project Manager/Scrum Master), Alex Rodriguez (Senior Full-Stack Developer), Emily Watson (Frontend Developer), Michael Kim (Backend Developer), Olivia Martinez (QA Engineer/DevOps Specialist), Liam Foster (UI/UX Designer)\n\n## I. Introduction and Team Introductions\n\nSarah Chen opened the meeting by welcoming the team to the first Sprint Planning meeting for the HealthTrack Pro project. Each team member introduced themselves and their roles.\n\n## II. Project Overview and Sprint Context\n\nSarah provided an overview of the HealthTrack Pro project:\n- HealthTrack Pro is a c

In [247]:
summary_agent.approval(summary_agent.state)

--- APPROVAL NODE ---
{'company_data': '# TechNova Solutions\n\n## Company Overview\nTechNova Solutions is a small, dynamic IT company specializing in web application development. With a team of 6 skilled professionals, they focus on creating innovative, user-friendly web solutions for small to medium-sized businesses.\n\n## Current Project: HealthTrack Pro\nTechNova is developing HealthTrack Pro, a comprehensive web application for personal health management. This application allows users to track their daily activities, nutrition, and health metrics, and provides insights and recommendations for a healthier lifestyle.\n\n## Team Structure\n1. ** Sarah Chen - Project Manager / Scrum Master**\n   - Oversees project progress, manages timelines, and facilitates communication\n   - Has a background in both frontend and backend development\n\n2. ** Alex Rodriguez - Senior Full-Stack Developer**\n   - Leads technical decisions and architecture design\n   - Proficient in both frontend and ba

{'company_data': '# TechNova Solutions\n\n## Company Overview\nTechNova Solutions is a small, dynamic IT company specializing in web application development. With a team of 6 skilled professionals, they focus on creating innovative, user-friendly web solutions for small to medium-sized businesses.\n\n## Current Project: HealthTrack Pro\nTechNova is developing HealthTrack Pro, a comprehensive web application for personal health management. This application allows users to track their daily activities, nutrition, and health metrics, and provides insights and recommendations for a healthier lifestyle.\n\n## Team Structure\n1. ** Sarah Chen - Project Manager / Scrum Master**\n   - Oversees project progress, manages timelines, and facilitates communication\n   - Has a background in both frontend and backend development\n\n2. ** Alex Rodriguez - Senior Full-Stack Developer**\n   - Leads technical decisions and architecture design\n   - Proficient in both frontend and backend technologies\n\n

In [252]:
summary_agent.state["summary_final"] = summary_agent.state["general_summary_draft"]
summary_agent.state["project_state"] = ""

In [253]:
summary_agent.state

{'company_data': '# TechNova Solutions\n\n## Company Overview\nTechNova Solutions is a small, dynamic IT company specializing in web application development. With a team of 6 skilled professionals, they focus on creating innovative, user-friendly web solutions for small to medium-sized businesses.\n\n## Current Project: HealthTrack Pro\nTechNova is developing HealthTrack Pro, a comprehensive web application for personal health management. This application allows users to track their daily activities, nutrition, and health metrics, and provides insights and recommendations for a healthier lifestyle.\n\n## Team Structure\n1. ** Sarah Chen - Project Manager / Scrum Master**\n   - Oversees project progress, manages timelines, and facilitates communication\n   - Has a background in both frontend and backend development\n\n2. ** Alex Rodriguez - Senior Full-Stack Developer**\n   - Leads technical decisions and architecture design\n   - Proficient in both frontend and backend technologies\n\n

In [254]:
summary_agent.update_project_state(summary_agent.state)

--- UPDATE PROJECT STATE NODE ---
{'company_data': '# TechNova Solutions\n\n## Company Overview\nTechNova Solutions is a small, dynamic IT company specializing in web application development. With a team of 6 skilled professionals, they focus on creating innovative, user-friendly web solutions for small to medium-sized businesses.\n\n## Current Project: HealthTrack Pro\nTechNova is developing HealthTrack Pro, a comprehensive web application for personal health management. This application allows users to track their daily activities, nutrition, and health metrics, and provides insights and recommendations for a healthier lifestyle.\n\n## Team Structure\n1. ** Sarah Chen - Project Manager / Scrum Master**\n   - Oversees project progress, manages timelines, and facilitates communication\n   - Has a background in both frontend and backend development\n\n2. ** Alex Rodriguez - Senior Full-Stack Developer**\n   - Leads technical decisions and architecture design\n   - Proficient in both fro

{'company_data': '# TechNova Solutions\n\n## Company Overview\nTechNova Solutions is a small, dynamic IT company specializing in web application development. With a team of 6 skilled professionals, they focus on creating innovative, user-friendly web solutions for small to medium-sized businesses.\n\n## Current Project: HealthTrack Pro\nTechNova is developing HealthTrack Pro, a comprehensive web application for personal health management. This application allows users to track their daily activities, nutrition, and health metrics, and provides insights and recommendations for a healthier lifestyle.\n\n## Team Structure\n1. ** Sarah Chen - Project Manager / Scrum Master**\n   - Oversees project progress, manages timelines, and facilitates communication\n   - Has a background in both frontend and backend development\n\n2. ** Alex Rodriguez - Senior Full-Stack Developer**\n   - Leads technical decisions and architecture design\n   - Proficient in both frontend and backend technologies\n\n

In [255]:
summary_agent.update_project_state_file(summary_agent.state)

--- UPDATE PROJECT STATE FILE NODE ---
Successfully exported Markdown to ../data_/project2.0/project-state.md


{'company_data': '# TechNova Solutions\n\n## Company Overview\nTechNova Solutions is a small, dynamic IT company specializing in web application development. With a team of 6 skilled professionals, they focus on creating innovative, user-friendly web solutions for small to medium-sized businesses.\n\n## Current Project: HealthTrack Pro\nTechNova is developing HealthTrack Pro, a comprehensive web application for personal health management. This application allows users to track their daily activities, nutrition, and health metrics, and provides insights and recommendations for a healthier lifestyle.\n\n## Team Structure\n1. ** Sarah Chen - Project Manager / Scrum Master**\n   - Oversees project progress, manages timelines, and facilitates communication\n   - Has a background in both frontend and backend development\n\n2. ** Alex Rodriguez - Senior Full-Stack Developer**\n   - Leads technical decisions and architecture design\n   - Proficient in both frontend and backend technologies\n\n

In [256]:
summary_agent.export_summary(summary_agent.state)

Successfully exported Markdown to ../data_/project2.0/memo/memo.md


{'company_data': '# TechNova Solutions\n\n## Company Overview\nTechNova Solutions is a small, dynamic IT company specializing in web application development. With a team of 6 skilled professionals, they focus on creating innovative, user-friendly web solutions for small to medium-sized businesses.\n\n## Current Project: HealthTrack Pro\nTechNova is developing HealthTrack Pro, a comprehensive web application for personal health management. This application allows users to track their daily activities, nutrition, and health metrics, and provides insights and recommendations for a healthier lifestyle.\n\n## Team Structure\n1. ** Sarah Chen - Project Manager / Scrum Master**\n   - Oversees project progress, manages timelines, and facilitates communication\n   - Has a background in both frontend and backend development\n\n2. ** Alex Rodriguez - Senior Full-Stack Developer**\n   - Leads technical decisions and architecture design\n   - Proficient in both frontend and backend technologies\n\n

In [258]:
tmp = summary_agent.state["process_message_history"]
summary_agent.state["process_message_history"] = ""
export_state(state=summary_agent.state,folder_path="../data_/project2.0/states/general_summary",filename="general_summary")
        